# Migration Flow in the US in 2017-2022, excluding 2020

The dataset for this project originates from the US Census Bureau. The dataset contains the State-to-State migration flows for 5 years from 2017 to 2022. The data has some interesting information about the population reside in each state, number of people resided in different state one year ago, and number of people was abroad one year ago.

## First glance at the dataset
The dataset has 5 Excel files which format in multiple headers and comments. The number of rows and columns in each data file is the same. 

## Exploring The Data
Here are some steps for exploring the data.
- Need to clean the dataset by removing all the titles and comments.
- Eliminate the MOE columns in all files.
- Set row 6 to be header and rename the column headers.
- Merge all the data files and transform to long format.

Let’s start with importing the necessary libaries, reading in the data, and  then aggregate.

In [1]:
# Import libaries
import pandas as pd
import numpy as np
import math
import glob
import re
import os

In [2]:
# Import all migration data
path = r'C:/Users/Nguyen/OneDrive/Desktop/migration-project/raw data/'
all_files = glob.glob(path + '/*.xlsx')

li = []

for filename in all_files:
    df = pd.read_excel(filename,na_values=['(NA)'], header = 6).fillna(0)
    
    # Remove all the MOE columns and the Footnotes which are from row 70 to 76 for all the data files
    df = df.loc[:70,(df != 'MOE').all()]

    # Adjust the column names
    df.rename(columns=
              {df.columns[0]: 'destination_state',
               df.columns[1]: 'population',
               df.columns[2]: 'same_house',
               df.columns[3]: 'same_state',
               df.columns[4]: 'from_different_state_Total',
               df.columns[-4]: 'abroad_Total',
               df.columns[-3]: 'abroad_PuertoRico',
               df.columns[-2]: 'abroad_USIslandArea',
               df.columns[-1]: 'abroad_ForeignCountry'
              }, inplace = True)

    #for i in range(4,56):
    #    df.rename(columns = {df.columns[i]:'from_state_' + df.columns[i]}, inplace = True)

    # Remove NA rows and duplicate columns
    df = df[df['destination_state'] != 0]
    df = df[df.columns.drop(list(df.filter(regex='Unnamed:')))]
    df = df.drop([2])
    df = df.drop([37])

    # Rename the first element and set the first column as index
    # df.values[0][0] = 'United States'
    
    # Change all columns to integer
    for col in df.columns[1:]:
        df[col] = df[col].astype('int64')
        
    # Create a column to store the year based on the Excel file name
    df['year'] = re.findall('\d+', os.path.basename(filename))[0]
    
    # Reset index
    df.reset_index(drop = True, inplace = True)
        
    li.append(df)

# Check out the first 5 rows of one of the files
li[0].head()

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year
0,Alabama,4819343,4158224,535649,111803,0,424,1513,517,2328,...,4277,1289,1070,1121,104,13667,476,0,13191,2017
1,Alaska,730874,595804,98168,30199,942,0,1387,203,3556,...,360,3653,0,2500,6,6703,0,965,5738,2017
2,Arizona,6935358,5717189,911120,261727,1337,2255,0,1763,59233,...,3593,14392,318,3897,997,45322,438,91,44793,2017
3,Arkansas,2968395,2509717,370309,79411,574,241,1060,0,5334,...,1087,1539,0,958,12,8958,573,0,8385,2017
4,California,39085984,34009331,4234770,523131,3715,2606,26907,3471,0,...,21162,33143,789,9348,1395,318752,1864,842,316046,2017


In [3]:
li[4].head()

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year
0,Alabama,5022366,4397470,471530,139263,0,714,1202,1536,8012,...,3654,1639,268,203,0,14103,311,611,13181,2022
1,Alaska,724196,614091,68351,36563,151,0,1645,10,3855,...,345,3076,0,943,42,5191,129,265,4797,2022
2,Arizona,7285247,6264863,690856,282729,727,2091,0,1147,74157,...,3791,21242,471,6569,1863,46799,608,840,45351,2022
3,Arkansas,3007872,2618120,294640,86375,1911,64,1776,0,7783,...,978,2703,43,481,326,8737,18,0,8719,2022
4,California,38629179,34335186,3515077,475803,5204,2040,27412,963,0,...,15373,31866,42,4407,1083,303113,1804,3158,298151,2022


In [85]:
li[0].head(25)

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year
0,Alabama,4819343,4158224,535649,111803,0,424,1513,517,2328,...,4277,1289,1070,1121,104,13667,476,0,13191,2017
1,Alaska,730874,595804,98168,30199,942,0,1387,203,3556,...,360,3653,0,2500,6,6703,0,965,5738,2017
2,Arizona,6935358,5717189,911120,261727,1337,2255,0,1763,59233,...,3593,14392,318,3897,997,45322,438,91,44793,2017
3,Arkansas,2968395,2509717,370309,79411,574,241,1060,0,5334,...,1087,1539,0,958,12,8958,573,0,8385,2017
4,California,39085984,34009331,4234770,523131,3715,2606,26907,3471,0,...,21162,33143,789,9348,1395,318752,1864,842,316046,2017
5,Colorado,5542282,4546729,734182,226933,3120,2030,6765,1356,27014,...,6099,6675,42,3506,5135,34438,828,568,33042,2017
6,Connecticut,3556297,3136666,311726,83325,206,540,644,160,2807,...,1314,374,0,188,257,24580,3780,0,20800,2017
7,Delaware,952449,830905,78998,38532,506,0,76,0,580,...,1510,0,371,296,0,4014,260,66,3688,2017
8,District of Columbia,684810,554225,64013,56231,494,0,549,105,3763,...,7509,206,0,115,0,10341,356,78,9907,2017
9,Florida,20772371,17554348,2402381,566476,9389,1400,5577,3028,30919,...,28232,6176,2187,7202,396,249166,28538,2847,217781,2017


In [74]:
all_files

['C:/Users/Nguyen/OneDrive/Desktop/migration-project/raw data\\State_to_State_Migrations_Table_2017.xlsx',
 'C:/Users/Nguyen/OneDrive/Desktop/migration-project/raw data\\State_to_State_Migrations_Table_2018.xlsx',
 'C:/Users/Nguyen/OneDrive/Desktop/migration-project/raw data\\State_to_State_Migrations_Table_2019.xlsx',
 'C:/Users/Nguyen/OneDrive/Desktop/migration-project/raw data\\State_to_State_Migrations_Table_2021.xlsx',
 'C:/Users/Nguyen/OneDrive/Desktop/migration-project/raw data\\State_to_State_Migrations_Table_2022.xlsx']

Concatenate all dataframes.

In [87]:
migration_df = pd.concat(li)
migration_df.head(56)

,destination_state,population,same_house,same_state,from_different_state_Total,Alabama,Alaska,Arizona,Arkansas,California,...,Virginia,Washington,West Virginia,Wisconsin,Wyoming,abroad_Total,abroad_PuertoRico,abroad_USIslandArea,abroad_ForeignCountry,year
0,Alabama,4819343,4158224,535649,111803,0,424,1513,517,2328,...,4277,1289,1070,1121,104,13667,476,0,13191,2017
1,Alaska,730874,595804,98168,30199,942,0,1387,203,3556,...,360,3653,0,2500,6,6703,0,965,5738,2017
2,Arizona,6935358,5717189,911120,261727,1337,2255,0,1763,59233,...,3593,14392,318,3897,997,45322,438,91,44793,2017
3,Arkansas,2968395,2509717,370309,79411,574,241,1060,0,5334,...,1087,1539,0,958,12,8958,573,0,8385,2017
4,California,39085984,34009331,4234770,523131,3715,2606,26907,3471,0,...,21162,33143,789,9348,1395,318752,1864,842,316046,2017
5,Colorado,5542282,4546729,734182,226933,3120,2030,6765,1356,27014,...,6099,6675,42,3506,5135,34438,828,568,33042,2017
6,Connecticut,3556297,3136666,311726,83325,206,540,644,160,2807,...,1314,374,0,188,257,24580,3780,0,20800,2017
7,Delaware,952449,830905,78998,38532,506,0,76,0,580,...,1510,0,371,296,0,4014,260,66,3688,2017
8,District of Columbia,684810,554225,64013,56231,494,0,549,105,3763,...,7509,206,0,115,0,10341,356,78,9907,2017
9,Florida,20772371,17554348,2402381,566476,9389,1400,5577,3028,30919,...,28232,6176,2187,7202,396,249166,28538,2847,217781,2017


Transform to long format.

In [88]:
id_var = ['destination_state','year', 'population','same_house', 'same_state','from_different_state_Total','abroad_Total']
id_values = [col for col in migration_df.columns if col not in id_var]
df_long = pd.melt( migration_df, id_var , id_values, 'from', 'number_of_people' )
df_long.head(53)

,destination_state,year,population,same_house,same_state,from_different_state_Total,abroad_Total,from,number_of_people
0,Alabama,2017,4819343,4158224,535649,111803,13667,Alabama,0
1,Alaska,2017,730874,595804,98168,30199,6703,Alabama,942
2,Arizona,2017,6935358,5717189,911120,261727,45322,Alabama,1337
3,Arkansas,2017,2968395,2509717,370309,79411,8958,Alabama,574
4,California,2017,39085984,34009331,4234770,523131,318752,Alabama,3715
5,Colorado,2017,5542282,4546729,734182,226933,34438,Alabama,3120
6,Connecticut,2017,3556297,3136666,311726,83325,24580,Alabama,206
7,Delaware,2017,952449,830905,78998,38532,4014,Alabama,506
8,District of Columbia,2017,684810,554225,64013,56231,10341,Alabama,494
9,Florida,2017,20772371,17554348,2402381,566476,249166,Alabama,9389


Export to CSV file and use this data to make Dashboard.

In [77]:
df_long.to_csv('migration_5years.csv', index=False)